Ноутбук запускался в колабе

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using Tesla T4 GPUs


In [2]:
! pip install datasets transformers seqeval

In [3]:
task = "ner"
batch_size = 8

# Data - PAD

In [4]:
!gdown --id 1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ

Downloading...
From: https://drive.google.com/uc?id=1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ
To: /content/NER_PAD_agg_test.csv
2.92MB [00:00, 95.3MB/s]


In [5]:
!gdown --id 1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t

Downloading...
From: https://drive.google.com/uc?id=1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t
To: /content/NER_PAD_agg_train.csv
11.7MB [00:00, 54.4MB/s]


In [6]:
!gdown --id 1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV

Downloading...
From: https://drive.google.com/uc?id=1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV
To: /content/NER_PAD_agg.csv
14.6MB [00:00, 55.1MB/s]


In [7]:
import pandas as pd
data = pd.read_csv('NER_PAD_agg.csv', converters={'tokens': eval, 'ner_tags': eval})

In [8]:
all_tags = set()
for _, row in data.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))

In [9]:
tag_to_idx = {t: i for i, t in enumerate(all_tags)}

In [10]:
label_list = {i: t for i, t in enumerate(all_tags)}

In [11]:
train = pd.read_csv('NER_PAD_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_PAD_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

In [12]:
train.ner_tags = train.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])
test.ner_tags = test.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])

In [13]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [14]:
from datasets import load_dataset, load_metric

In [15]:
datasets = load_dataset('pandas', data_files={'train' : 'train.pkl', 'test' : 'test.pkl'})

Using custom data configuration default-28cbf0b91c6f634d


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-28cbf0b91c6f634d/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 800
    })
})

# Tokenizer

In [17]:
from transformers import AutoTokenizer
model_checkpoint = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [18]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [19]:
label_all_tokens = False

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Warning : есть последовательности больше, чем максимально возможная в BERT - 512

In [22]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    3200.000000
mean      348.981875
std       236.542223
min         6.000000
25%       151.750000
50%       308.000000
75%       434.000000
max      4288.000000
dtype: float64

In [23]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count     800.000000
mean      347.810000
std       230.390747
min        36.000000
25%       145.750000
50%       307.500000
75%       431.750000
max      1471.000000
dtype: float64

In [24]:
train_tokenized = tokenized_datasets["train"].to_pandas()
test_tokenized = tokenized_datasets["test"].to_pandas()

In [25]:
train_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",SV990115834_ОРГ__INM_16'04'09_3.pdf,"[101, 254, 3274, 874, 144, 130, 274, 94901, 11...","[-100, 15, 15, -100, 15, -100, 15, 15, -100, -...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[l, испр, 4-, v, moscow, quarterly, report, li..."


In [26]:
test_tokenized.head(1)

,attention_mask,id,input_ids,labels,ner_tags,token_type_ids,tokens
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",B31005082_ОРГ__INM_15'39'47_1.pdf,"[101, 254, 867, 106, 32877, 905, 138, 34193, 1...","[-100, 15, 15, 15, 15, 15, 15, 15, 15, 15, -10...","[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[l, т, !, приложение, №, 1, постановлению, пра..."


In [27]:
CHUNCK_SIZE = 512

def lists_to_chuncks(row):
    n = CHUNCK_SIZE - 2
    new_row = row.copy()
    for key in ['attention_mask', 'input_ids', 'labels', 'token_type_ids']:
        FIRST = new_row[key][0]
        LAST = new_row[key][-1]
        new_row[key] = new_row[key][1:-1]
        new_row[key] = [[FIRST] + list(new_row[key][i:i + n]) + [LAST] for i in range(0, len(new_row[key]), n)]
    return new_row

In [28]:
train_tokenized = train_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
train_tokenized = train_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [29]:
test_tokenized = test_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
test_tokenized = test_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [30]:
train_tokenized.to_pickle('train_tokenized.pkl')
test_tokenized.to_pickle('test_tokenized.pkl')

In [31]:
tokenized_datasets = load_dataset('pandas', data_files={'train' : 'train_tokenized.pkl', 'test' : 'test_tokenized.pkl'})

Using custom data configuration default-f47a3ae22b8021dc


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-f47a3ae22b8021dc/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [32]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    3778.000000
mean      295.896771
std       145.507633
min         4.000000
25%       145.250000
50%       294.000000
75%       421.000000
max       512.000000
dtype: float64

In [33]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count    943.000000
mean     295.370095
std      144.487497
min        3.000000
25%      143.500000
50%      295.000000
75%      419.500000
max      512.000000
dtype: float64

Теперь все последовательности нужной длины

# Model

In [34]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Warning : модель надо обучить на наших данных

In [35]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [36]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [37]:
metric = load_metric("seqeval")

In [38]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.053952,0.763320,0.766651,0.764982,0.984274
2,0.123500,0.039918,0.845012,0.836472,0.840720,0.988903
3,0.045600,0.039912,0.824781,0.863803,0.843841,0.988681


TrainOutput(global_step=1419, training_loss=0.06902886109086642, metrics={'train_runtime': 1358.5511, 'train_samples_per_second': 1.044, 'total_flos': 1732874538208512.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 2009686016, 'init_mem_gpu_alloc_delta': 709382656, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 78442496, 'train_mem_gpu_alloc_delta': 2197097472, 'train_mem_cpu_peaked_delta': 334073856, 'train_mem_gpu_peaked_delta': 6504154112})

In [41]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9886814905010308,
 'eval_f1': 0.8438411487547677,
 'eval_loss': 0.03991246595978737,
 'eval_mem_cpu_alloc_delta': 671744,
 'eval_mem_cpu_peaked_delta': 1241088,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 417779712,
 'eval_precision': 0.824780701754386,
 'eval_recall': 0.8638033991731741,
 'eval_runtime': 36.5174,
 'eval_samples_per_second': 25.823}

In [42]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'DOCAGRDATE': {'f1': 0.7065527065527065,
  'number': 150,
  'precision': 0.6169154228855721,
  'recall': 0.8266666666666667},
 'DOCAGRNUM': {'f1': 0.5842696629213483,
  'number': 179,
  'precision': 0.5875706214689266,
  'recall': 0.5810055865921788},
 'DOCAMOUNT': {'f1': 0.8644859813084113,
  'number': 1243,
  'precision': 0.8377358490566038,
  'recall': 0.8930008045052292},
 'DOCCPTY': {'f1': 0.8562644119907763,
  'number': 595,
  'precision': 0.7889518413597734,
  'recall': 0.9361344537815126},
 'DOCCPTYINN': {'f1': 0.9261744966442953,
  'number': 282,
  'precision': 0.8789808917197452,
  'recall': 0.9787234042553191},
 'DOCCUSTOMER': {'f1': 0.9247822644497229,
  'number': 609,
  'precision': 0.8929663608562691,
  'recall': 0.9589490968801314},
 'DOCDATE': {'f1': 0.8726731198808638,
  'number': 645,
  'precision': 0.839541547277937,
  'recall': 0.9085271317829458},
 'DOCNUM': {'f1': 0.73943661971831,
  'number': 651,
  'precision': 0.865979381443299,
  'recall': 0.6451612903225806}

In [43]:
pd.DataFrame.from_dict(results).transpose().iloc[:-4, :-1]

,precision,recall,f1
DOCAGRDATE,0.616915,0.826667,0.706553
DOCAGRNUM,0.587571,0.581006,0.584270
DOCAMOUNT,0.837736,0.893001,0.864486
DOCCPTY,0.788952,0.936134,0.856264
DOCCPTYINN,0.878981,0.978723,0.926174
DOCCUSTOMER,0.892966,0.958949,0.924782
DOCDATE,0.839542,0.908527,0.872673
DOCNUM,0.865979,0.645161,0.739437


In [44]:
pd.DataFrame.from_dict(results).transpose().iloc[-4:, 0].rename('result')

overall_precision    0.824781
overall_recall       0.863803
overall_f1           0.843841
overall_accuracy     0.988681
Name: result, dtype: float64

In [45]:
!pip install sklearn_crfsuite

In [46]:
from sklearn_crfsuite import metrics
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags, digits=3))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

 B-DOCAGRDATE      0.656     0.827     0.732       150
  B-DOCAGRNUM      0.627     0.620     0.624       179
  B-DOCAMOUNT      0.861     0.899     0.879      1243
    B-DOCCPTY      0.837     0.938     0.884       595
 B-DOCCPTYINN      0.879     0.979     0.926       282
B-DOCCUSTOMER      0.933     0.956     0.944       609
    B-DOCDATE      0.890     0.936     0.912       645
     B-DOCNUM      0.866     0.645     0.739       651
 I-DOCAGRDATE      0.717     0.864     0.784        88
  I-DOCAGRNUM      0.000     0.000     0.000         7
  I-DOCAMOUNT      0.799     0.869     0.832       620
    I-DOCCPTY      0.833     0.949     0.887       564
I-DOCCUSTOMER      0.886     0.961     0.922       462
    I-DOCDATE      0.945     0.936     0.940       605
     I-DOCNUM      0.000     0.000     0.000         0
            O      0.995     0.994     0.994    137842

    micro avg      0.989     0.989     0.989    144542
    macr

In [47]:
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags[:-1], digits=3))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

 B-DOCAGRDATE      0.656     0.827     0.732       150
  B-DOCAGRNUM      0.627     0.620     0.624       179
  B-DOCAMOUNT      0.861     0.899     0.879      1243
    B-DOCCPTY      0.837     0.938     0.884       595
 B-DOCCPTYINN      0.879     0.979     0.926       282
B-DOCCUSTOMER      0.933     0.956     0.944       609
    B-DOCDATE      0.890     0.936     0.912       645
     B-DOCNUM      0.866     0.645     0.739       651
 I-DOCAGRDATE      0.717     0.864     0.784        88
  I-DOCAGRNUM      0.000     0.000     0.000         7
  I-DOCAMOUNT      0.799     0.869     0.832       620
    I-DOCCPTY      0.833     0.949     0.887       564
I-DOCCUSTOMER      0.886     0.961     0.922       462
    I-DOCDATE      0.945     0.936     0.940       605
     I-DOCNUM      0.000     0.000     0.000         0

    micro avg      0.856     0.888     0.872      6700
    macro avg      0.715     0.758     0.734      6700
 weighte